In [95]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

import sklearn.metrics
from sklearn.model_selection import GridSearchCV

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
# render plot in default browser
pio.renderers.default = 'browser'

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


### Reading in data with raw values and adding GDP
This mainly applies to Redfin data- rather than a percentage use the raw numbers

In [96]:
data = pd.read_pickle('dataraw_to2018.pkl')
gdp = pd.read_csv('GDP.csv')

gdp.head()

,DATE,GDP
0,1947-01-01,243.164
1,1947-04-01,245.968
2,1947-07-01,249.585
3,1947-10-01,259.745
4,1948-01-01,265.742


In [97]:
gdp['year'] = gdp['DATE'].apply(lambda x: x[:4])
gdp['month'] = gdp['DATE'].apply(lambda x: x[5:7])
gdp.head()

,DATE,GDP,year,month
0,1947-01-01,243.164,1947,01
1,1947-04-01,245.968,1947,04
2,1947-07-01,249.585,1947,07
3,1947-10-01,259.745,1947,10
4,1948-01-01,265.742,1948,01


In [98]:
gdp = gdp[gdp['month']=='10']

In [99]:
gdp = gdp[['year', 'GDP']]
gdp['year'] = gdp['year'].apply(lambda x: int(x))

In [100]:
data.shape

(3072, 79)

In [101]:
data = data.merge(gdp, on='year', how='left')

### Train/Test/Val
Split data through 2018 into X and y then split 75/25 train/test. 2019 later is used as validation.

In [102]:
X = data.iloc[:,3:-2].join(data.iloc[:,-1])
y = data.iloc[:,-2:-1]

In [103]:
#X_norm = StandardScaler().fit_transform(X)
X_norm = X
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.25, random_state=42)


### Model Training and Selection
Extra Trees Regressor returns the best R2 on test and validation data

In [104]:
regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(X_train, y_train)

/var/folders/j2/51tvlt7n02xc69lkwdrbspqw0000gn/T/ipykernel_11968/2848552734.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(max_depth=10, random_state=0)

In [105]:
score = regr.score(X_test, y_test)
print("Random Forest Regressor Test score: "+str(score)+'\n')

##0.5669654667807917

Random Forest Regressor Test score: 0.566822295903957



In [106]:
# parameters = {'n_estimators': [100, 500, 1000],
#                 'max_depth':[5, 10, 15], 
#                 'max_features': ['auto', 'sqrt', 'log2'],
#                 'bootstrap': [True, False]}
# model = RandomForestRegressor()
# grid = GridSearchCV(model, parameters)
# grid.fit(X, y)

In [107]:
# grid.cv_results_

In [108]:
# grid.best_estimator_

## RandomForestRegressor(bootstrap=False, max_depth=15, max_features='sqrt',
 ##                     n_estimators=1000)

In [109]:
regr = RandomForestRegressor(bootstrap=False, max_depth=15, 
                                max_features='sqrt', n_estimators=1000,
                                random_state=0)
regr.fit(X_train, y_train)

/var/folders/j2/51tvlt7n02xc69lkwdrbspqw0000gn/T/ipykernel_11968/2106608459.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=False, max_depth=15, max_features='sqrt',
                      n_estimators=1000, random_state=0)

In [110]:
score = regr.score(X_test, y_test)
print("Random Forest Regressor Test score: "+str(score)+'\n')

##0.5554493055329761

Random Forest Regressor Test score: 0.5552637737365926



In [111]:
##try extra trees regressor
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(n_estimators=100, random_state=0).fit(
       X_train, y_train)
et.score(X_test, y_test)

## 0.6272394296098471

/var/folders/j2/51tvlt7n02xc69lkwdrbspqw0000gn/T/ipykernel_11968/4119476172.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.6284935346003326

In [112]:
# parameters = {'n_estimators': [100, 500, 1000],
#                 'max_depth':[5, 10, 15], 
#                 'max_features': ['auto', 'sqrt', 'log2'],
#                 'bootstrap': [True, False]}
# model = ExtraTreesRegressor()
# grid_et = GridSearchCV(model, parameters)
# grid_et.fit(X, y)

/Users/jennyney/Dev/Capstone/capstone_git/capstone_env_v2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/Users/jennyney/Dev/Capstone/capstone_git/capstone_env_v2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/Users/jennyney/Dev/Capstone/capstone_git/capstone_env_v2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/Users/jennyney/Dev/Capstone/capstone_git/capstone_env_v2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:531: DataConve

GridSearchCV(estimator=ExtraTreesRegressor(),
             param_grid={'bootstrap': [True, False], 'max_depth': [5, 10, 15],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 500, 1000]})

In [113]:
grid_et.best_estimator_

##ExtraTreesRegressor(max_depth=15, n_estimators=500)

ExtraTreesRegressor(bootstrap=True, max_depth=15, n_estimators=1000)

In [121]:
# et = ExtraTreesRegressor(n_estimators=500, max_depth=15, random_state=0).fit(
#        X_train, y_train)
et = ExtraTreesRegressor(max_depth=15, n_estimators=500, random_state=0).fit(
       X_train, y_train)

et.score(X_test, y_test)

## 0.6294494812587801

/var/folders/j2/51tvlt7n02xc69lkwdrbspqw0000gn/T/ipykernel_11968/198603156.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.6294600060880793

In [122]:
df2019 = pd.read_pickle('dataraw_2019.pkl')
df2019.head()

,state_fips,county_fips,year,debt_ratio_low,debt_ratio_high,corp_income_tax_low,corp_income_tax_high,income_tax_low,income_tax_high,11_avg_annual_employee_pct_chg,...,hs_diploma_pct,some_college_lessthan_1yr_pct,some_college_greaterthan_1yr_pct,bachelor_degree_pct,master_degree_pct,professional_degree_pct,doctorate_degree_pct,occupied_units_pct,vacant_units_pct,annual_change_pct
0,1.0,01015,2019,1.2800,1.475,6.5,6.5,2.0,5.0,0.000000,...,0.197808,0.036301,0.115479,0.079944,0.034479,0.009471,0.010404,0.829527,0.170473,3.43
1,1.0,01043,2019,1.6400,1.905,6.5,6.5,2.0,5.0,0.000000,...,0.162496,0.055355,0.109779,0.070850,0.033366,0.004083,0.002017,0.838823,0.161177,4.19
2,1.0,01049,2019,1.1450,1.325,6.5,6.5,2.0,5.0,100.000000,...,0.196132,0.048131,0.072840,0.051641,0.029072,0.003174,0.000000,0.787192,0.212808,4.22
3,1.0,01055,2019,1.5275,1.760,6.5,6.5,2.0,5.0,0.000000,...,0.179714,0.061202,0.117769,0.073571,0.030176,0.009074,0.008126,0.836966,0.163034,3.60
4,1.0,01073,2019,0.8950,1.100,6.5,6.5,2.0,5.0,1.408451,...,0.156648,0.038477,0.105712,0.139755,0.061027,0.023885,0.012866,0.854944,0.145056,3.75


In [123]:
gdp['GDP'][gdp['year']==2019]

291    21694.458
Name: GDP, dtype: float64

In [124]:
df2019['GDP'] = 21694.458

In [125]:
#X_val = StandardScaler().fit_transform(df2019.iloc[:,3:-2].join(df2019.iloc[:,-1]))
X_val = df2019.iloc[:,3:-2].join(df2019.iloc[:,-1])
y_val = df2019.iloc[:,-2:-1]

In [89]:
score = regr.score(X_val, y_val)
print("Random Forest Regressor Validation score: "+str(score)+'\n')

## -0.16832835442893224

Random Forest Regressor Validation score: -0.06228040267627066



In [126]:
et.score(X_val, y_val)

## -0.00022179764539975722
##no scaler == 0.2589469340774412

0.2589469340774412

In [91]:
d = dict()
for i, j in zip(X.columns, regr.feature_importances_):
    d[i]=j

print({k: v for k, v in sorted(d.items(), key=lambda item: item[1])})
#print(et.feature_importances_)

{'21_avg_annual_pay_pct_chg': 0.0036844771224022774, 'birth_45_50_pct': 0.004026009276510636, '21_avg_annual_employee_pct_chg': 0.004152944048024643, '11_avg_annual_pay_pct_chg': 0.004405767621757717, '11_avg_annual_employee_pct_chg': 0.004811253297107085, 'birth_15_19_pct': 0.00489752956793322, '99_avg_annual_pay_pct_chg': 0.00509643810051376, '42_avg_annual_pay_pct_chg': 0.005269327612092776, '71_avg_annual_pay_pct_chg': 0.005314067918365244, '55_avg_annual_pay_pct_chg': 0.00536570865108804, '22_avg_annual_employee_pct_chg': 0.005426338148432276, '22_avg_annual_pay_pct_chg': 0.00543219148660849, '62_avg_annual_pay_pct_chg': 0.005438256432847808, '81_avg_annual_pay_pct_chg': 0.005494735667695048, '56_avg_annual_employee_pct_chg': 0.00556315523880675, '54_avg_annual_pay_pct_chg': 0.005570140144384466, '53_avg_annual_employee_pct_chg': 0.005638710575002025, '52_avg_annual_pay_pct_chg': 0.0056908932913927405, '55_avg_annual_employee_pct_chg': 0.005720177431246267, '51_avg_annual_pay_pct_

In [127]:
d = dict()
for i, j in zip(X.columns, et.feature_importances_):
    d[i]=j

print({k: v for k, v in sorted(d.items(), key=lambda item: item[1])})

{'21_avg_annual_pay_pct_chg': 0.0028170402529790555, '11_avg_annual_pay_pct_chg': 0.0030687587988049362, '62_avg_annual_pay_pct_chg': 0.0031788900370751457, '61_avg_annual_pay_pct_chg': 0.0034122837620939642, '61_avg_annual_employee_pct_chg': 0.003499749383355173, '21_avg_annual_employee_pct_chg': 0.0036522751264605044, '22_avg_annual_employee_pct_chg': 0.0037246175511805564, '81_avg_annual_pay_pct_chg': 0.0037676379217616764, '71_avg_annual_pay_pct_chg': 0.0039113932413558155, '51_avg_annual_pay_pct_chg': 0.004101866353943255, '11_avg_annual_employee_pct_chg': 0.004130450372875946, '54_avg_annual_pay_pct_chg': 0.004250340850481281, '22_avg_annual_pay_pct_chg': 0.0042581017377707146, '55_avg_annual_employee_pct_chg': 0.004305053314657646, '92_avg_annual_pay_pct_chg': 0.004421059732738451, '23_avg_annual_pay_pct_chg': 0.004500195144466099, '56_avg_annual_pay_pct_chg': 0.004518483546963503, '71_avg_annual_employee_pct_chg': 0.004602349059357196, '51_avg_annual_employee_pct_chg': 0.004620

In [134]:
d = {'Feature': X.columns, 'Importance': et.feature_importances_}
df = pd.DataFrame(d)
df = df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

fig = px.bar_polar(df.iloc[:30,:], r='Importance', theta='Feature',
            color='Feature', template='plotly_dark',
            color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.show()

In [130]:
df[df['Importance']>0.0035]

,Feature,Importance
5,income_tax_high,0.106714
49,months_of_supply,0.078779
75,GDP,0.065205
4,income_tax_low,0.058478
3,corp_income_tax_high,0.051453
...,...,...
29,51_avg_annual_pay_pct_chg,0.004102
37,71_avg_annual_pay_pct_chg,0.003911
39,81_avg_annual_pay_pct_chg,0.003768
8,22_avg_annual_employee_pct_chg,0.003725


In [135]:
pred2019 = et.predict(X_val)


df2019['Predicted_HPI_change'] = pred2019

df2019['Prediction_delta'] = ((df2019['annual_change_pct'] - df2019['Predicted_HPI_change'])/df2019['annual_change_pct'])*100
print(df2019['Prediction_delta'].mean())
#5.5799%
print(df2019['Prediction_delta'].median())

11.071360602842423
-8.914948446736185


In [94]:

fig = px.choropleth(df2019, geojson=counties, locations='county_fips', color='Prediction_delta',
                           color_continuous_scale="Viridis",
                            range_color=(0, 100),
                           scope="usa",
                           labels={'Prediction_delta':'Prediction delta for 2019 HPI'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()